In [1]:
import warnings
warnings.filterwarnings('ignore',category=FutureWarning)
warnings.filterwarnings('ignore',category=DeprecationWarning)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from functools import reduce
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [2]:
path = "../Our_Dataset"
files_list = []
for filename in os.listdir(path):
    f = os.path.join(path, filename)
        # checking if it is a file
    if os.path.isfile(f):
        files_list.append(f)
print("Number of files: {}".format(len(files_list)))

Number of files: 11


## Merge Dataset

In [3]:
i = 0
df = pd.DataFrame()
drop_list = ['Data','Hora', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5']
while i < len(files_list):
    print(files_list[i])
    if(files_list[i] == "../Our_Dataset/PT0002000052213942JN.xlsx" or files_list[i] == "../Our_Dataset/PT0002000077861448JB.xlsx" or files_list[i] == "../Our_Dataset/PT0002000009223751SH.xlsx" or files_list[i] == "../Our_Dataset/PT0002000077784987RW.xlsx"):
        i += 1
        continue
    data = pd.read_excel(files_list[i], sheet_name=['Total_Consumers'])
    df_tmp = data['Total_Consumers']
    df_tmp['Data'] = df_tmp['Data'].apply(lambda x: str(x).replace('/', '-'))
    df_tmp['Time'] = df_tmp['Data'] + ' ' + df_tmp['Hora']
    df_tmp['Time'] = pd.to_datetime(df_tmp['Time'], infer_datetime_format=True)
    tmp_filter = df_tmp.filter(drop_list)
    df_tmp.drop(tmp_filter, axis=1, inplace=True)
    df_tmp.dropna(subset=['Time'], inplace=True)
    df_tmp = df_tmp.iloc[:,[1,0]]
    df_tmp = df_tmp.drop_duplicates(subset=['Time'])
    print(df_tmp.shape)
    print("Time range" + ": " + str(min(df_tmp['Time'])) + " - " + str(max(df_tmp['Time'])))
    if len(df) == 0:
        df = df_tmp
    else:
        df = df.merge(df_tmp, on='Time', how='outer')
    name = files_list[i].split("/")[2]
    name = name[0:len(name)-5]
    name = name[0:2] + name[len(name)-4:]
    df.rename(columns={'Consumo Registado': name}, inplace=True)
    print(df.shape)
    i += 1
df

../Our_Dataset/PT0002000110339041CV.xlsx
(75448, 2)
Time range: 2020-10-06 00:15:00 - 2022-12-01 00:00:00
(75448, 2)
../Our_Dataset/PT0002000009906944AL.xlsx
(39260, 2)
Time range: 2021-10-19 00:15:00 - 2022-12-02 00:00:00
(75544, 3)
../Our_Dataset/PT0002000052213942JN.xlsx
../Our_Dataset/PT0002000105307115VP.xlsx
(37916, 2)
Time range: 2021-11-01 00:15:00 - 2022-12-01 00:00:00
(75544, 4)
../Our_Dataset/PT0002000051290369QG.xlsx
(37052, 2)
Time range: 2021-10-11 00:15:00 - 2022-11-01 00:00:00
(75544, 5)
../Our_Dataset/PT0002000077861448JB.xlsx
../Our_Dataset/PT0002000077784987RW.xlsx
../Our_Dataset/PT0002000071112543QL.xlsx
(69208, 2)
Time range: 2021-01-01 00:15:00 - 2022-12-23 00:00:00
(77560, 6)
../Our_Dataset/PT0002000009686817RW.xlsx
(81584, 2)
Time range: 2020-11-01 00:00:00 - 2023-02-28 23:45:00
(84087, 7)
../Our_Dataset/PT0002000009223751SH.xlsx
../Our_Dataset/PT0002000083005987ZW.xlsx
(74876, 2)
Time range: 2020-10-06 00:15:00 - 2022-11-25 01:00:00
(84087, 8)


,Time,PT41CV,PT44AL,PT15VP,PT69QG,PT43QL,PT17RW,PT87ZW
0,2020-10-06 00:15:00,0.076,NaN,NaN,NaN,NaN,NaN,0.320
1,2020-10-06 00:30:00,0.072,NaN,NaN,NaN,NaN,NaN,0.308
2,2020-10-06 00:45:00,0.072,NaN,NaN,NaN,NaN,NaN,0.292
3,2020-10-06 01:00:00,0.068,NaN,NaN,NaN,NaN,NaN,0.280
4,2020-10-06 01:15:00,0.032,NaN,NaN,NaN,NaN,NaN,0.340
...,...,...,...,...,...,...,...,...
84082,2023-02-28 22:45:00,NaN,NaN,NaN,NaN,NaN,0.064,NaN
84083,2023-02-28 23:00:00,NaN,NaN,NaN,NaN,NaN,0.064,NaN
84084,2023-02-28 23:15:00,NaN,NaN,NaN,NaN,NaN,0.064,NaN
84085,2023-02-28 23:30:00,NaN,NaN,NaN,NaN,NaN,0.064,NaN


In [4]:
df_sync = df.dropna()
df_sync.reset_index(inplace=True, drop=True)
print("Time range" + ": " + str(min(df_sync['Time'])) + " - " + str(max(df_sync['Time'])))
df_sync

Time range: 2021-11-01 00:15:00 - 2022-11-01 00:00:00


,Time,PT41CV,PT44AL,PT15VP,PT69QG,PT43QL,PT17RW,PT87ZW
0,2021-11-01 00:15:00,0.028,0.452,0.000,0.828,0.128,0.056,0.552
1,2021-11-01 00:30:00,0.080,0.440,0.000,0.796,0.160,0.056,0.492
2,2021-11-01 00:45:00,0.092,0.424,0.000,0.832,0.220,0.056,0.308
3,2021-11-01 01:00:00,0.040,0.408,0.000,0.800,0.404,0.088,0.156
4,2021-11-01 01:15:00,0.076,0.396,0.000,0.836,0.156,0.116,0.160
...,...,...,...,...,...,...,...,...
35027,2022-10-31 23:00:00,0.120,0.448,0.108,0.220,0.164,0.144,0.368
35028,2022-10-31 23:15:00,0.036,0.424,0.124,0.252,0.228,0.164,0.396
35029,2022-10-31 23:30:00,0.064,0.392,0.148,0.268,0.224,0.220,0.356
35030,2022-10-31 23:45:00,0.132,0.260,0.132,0.276,0.212,0.176,0.392


In [5]:
dplctd = df_sync.duplicated(subset=['Time']).sum()
dplctd

0

In [6]:
dplctd = df.duplicated(subset=['Time']).sum()
dplctd

0

In [7]:
df.sort_values(by=['Time'], inplace=True)
df

,Time,PT41CV,PT44AL,PT15VP,PT69QG,PT43QL,PT17RW,PT87ZW
0,2020-10-06 00:15:00,0.076,NaN,NaN,NaN,NaN,NaN,0.320
1,2020-10-06 00:30:00,0.072,NaN,NaN,NaN,NaN,NaN,0.308
2,2020-10-06 00:45:00,0.072,NaN,NaN,NaN,NaN,NaN,0.292
3,2020-10-06 01:00:00,0.068,NaN,NaN,NaN,NaN,NaN,0.280
4,2020-10-06 01:15:00,0.032,NaN,NaN,NaN,NaN,NaN,0.340
...,...,...,...,...,...,...,...,...
84082,2023-02-28 22:45:00,NaN,NaN,NaN,NaN,NaN,0.064,NaN
84083,2023-02-28 23:00:00,NaN,NaN,NaN,NaN,NaN,0.064,NaN
84084,2023-02-28 23:15:00,NaN,NaN,NaN,NaN,NaN,0.064,NaN
84085,2023-02-28 23:30:00,NaN,NaN,NaN,NaN,NaN,0.064,NaN


In [8]:
for feature in df_sync.columns:
    print(feature + ": " + str(min(df_sync[feature])) + " - " + str(max(df_sync[feature])))

Time: 2021-11-01 00:15:00 - 2022-11-01 00:00:00
PT41CV: 0.0 - 3.172
PT44AL: 0.0 - 6.076
PT15VP: 0.0 - 3.26
PT69QG: 0.0 - 3.58
PT43QL: 0.04 - 4.856
PT17RW: 0.0 - 3.748
PT87ZW: 0.0 - 5.656


In [9]:
df_test = df_sync.copy()
df_test = df_test.set_index("Time")
df_test

,PT41CV,PT44AL,PT15VP,PT69QG,PT43QL,PT17RW,PT87ZW
Time,,,,,,,
2021-11-01 00:15:00,0.028,0.452,0.000,0.828,0.128,0.056,0.552
2021-11-01 00:30:00,0.080,0.440,0.000,0.796,0.160,0.056,0.492
2021-11-01 00:45:00,0.092,0.424,0.000,0.832,0.220,0.056,0.308
2021-11-01 01:00:00,0.040,0.408,0.000,0.800,0.404,0.088,0.156
2021-11-01 01:15:00,0.076,0.396,0.000,0.836,0.156,0.116,0.160
...,...,...,...,...,...,...,...
2022-10-31 23:00:00,0.120,0.448,0.108,0.220,0.164,0.144,0.368
2022-10-31 23:15:00,0.036,0.424,0.124,0.252,0.228,0.164,0.396
2022-10-31 23:30:00,0.064,0.392,0.148,0.268,0.224,0.220,0.356


### Missing 8 values from 1:00 am to 3:00 am from 27/03/2022 in all sheets due to time change

In [10]:
df1 = df_test.asfreq('15T')
df1

,PT41CV,PT44AL,PT15VP,PT69QG,PT43QL,PT17RW,PT87ZW
Time,,,,,,,
2021-11-01 00:15:00,0.028,0.452,0.000,0.828,0.128,0.056,0.552
2021-11-01 00:30:00,0.080,0.440,0.000,0.796,0.160,0.056,0.492
2021-11-01 00:45:00,0.092,0.424,0.000,0.832,0.220,0.056,0.308
2021-11-01 01:00:00,0.040,0.408,0.000,0.800,0.404,0.088,0.156
2021-11-01 01:15:00,0.076,0.396,0.000,0.836,0.156,0.116,0.160
...,...,...,...,...,...,...,...
2022-10-31 23:00:00,0.120,0.448,0.108,0.220,0.164,0.144,0.368
2022-10-31 23:15:00,0.036,0.424,0.124,0.252,0.228,0.164,0.396
2022-10-31 23:30:00,0.064,0.392,0.148,0.268,0.224,0.220,0.356


In [11]:
inds = pd.isnull(df1.values).any(1).nonzero()[0]
inds

array([14019, 14020, 14021, 14022, 14023, 14024, 14025, 14026])

In [12]:
a = df1.iloc[inds[:]]
a

,PT41CV,PT44AL,PT15VP,PT69QG,PT43QL,PT17RW,PT87ZW
Time,,,,,,,
2022-03-27 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-03-27 01:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-03-27 01:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-03-27 01:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-03-27 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-03-27 02:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-03-27 02:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-03-27 02:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
df.to_csv("total_output.csv")

In [14]:
df_test.to_csv("output.csv")

## Merge with weather data

In [ ]:
df_weather = pd.read_csv("../")